# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets
from scipy.stats import norm

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
#alt.data_transformers.enable('data_server')
alt.data_transformers.disable_max_rows()
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [2]:
start_date = pd.to_datetime("2020-04-13")
end_date = pd.to_datetime("2020-11-06")

In [3]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [4]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [5]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0


In [6]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

,grupo_de_edad,fecha,fallecidos
28,0-39,2020-04-13,3
29,40-49,2020-04-13,2
30,50-59,2020-04-13,3
31,60-69,2020-04-13,12
32,70-79,2020-04-13,29


## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos la fecha por cada grupo etáreo.

In [7]:
alt.Chart(fallecidos_etareo).mark_bar().encode(
    x=alt.X('grupo_de_edad', axis=alt.Axis(title='Grupo etario')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Cantidad de fallecidos')),
    tooltip=[alt.Tooltip('fallecidos',title='Cantidad de fallecidos'), alt.Tooltip('fecha',title='Fecha ultima actualizacion')]
).properties(
    title='Fallecidos a la fecha por grupo etareo',
    width=500,
    height=300
    
)

alt.Chart(...)

**Comentarios:** A la fecha el grupo etareo con mayor cantidad de fallecidos es el que va de 70-79 años, y el que tiene menor cantidad es el grupo que va entre 0-39 años. 

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [8]:
alt.Chart(activos).transform_density(
    'poblacion',
    as_=['poblacion', 'density'],
).mark_area().encode(
    x=alt.X('poblacion', axis=alt.Axis(title='Poblacion')),
    y=alt.X('density:Q', axis=alt.Axis(title='Densidad')),
).properties(
    title='Distribucion de la poblacion por comuna',
    width= 1000,
    height= 500
)

alt.Chart(...)

In [9]:
promedio_poblacion=activos['poblacion'].mean()
desv_s_poblacion=activos['poblacion'].std()
print('El promedio de la poblacion es', promedio_poblacion, ', y la desviacion estandar es' ,desv_s_poblacion)

El promedio de la poblacion es 56237.89017341041 , y la desviacion estandar es 88819.28419046862


**Comentarios:** En el gráfico podemos ver la densidad, donde el area bajo la curva se interpreta como la cantidad de comunas con la determinada cantidad de población, por lo que podemos deducir que es más común encontrarse comunas con población desde 0 a 80000 habitantes, y a medida que vamos aumentando la población, disminuye la cantidad de comunas que cumplen esa cuota. En particular tenemos que el promedio de la poblacion ronda los 56237 por comuna, y la desviacion estandar es de 88819 aproximadamente, por lo que los datos varian mucho como se puede apreciar graficamente.

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [10]:
alt.Chart(fallecidos_etareo).mark_line(size=4).encode(
    x=alt.X('fecha', axis=alt.Axis(title='Mes')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Fallecidos')),
    color=alt.Color('grupo_de_edad', title='Grupo etario:'),
    # Que poner el mouse encima de la curva muestre la cantidad de fallecidos a la fecha, fecha del instante y grupo etario
    tooltip=[alt.Tooltip('fallecidos',title='Cantidad de fallecidos'), alt.Tooltip('fecha', title='Fecha actual'),
            alt.Tooltip('grupo_de_edad', title='Grupo etario')]
).properties(
    title='Evolucion temporal de la cantidad de fallecidos por grupo etario',
    width=500,
    height=400
)

alt.Chart(...)

**Comentarios:** Desprendemos que el grupo de 70-79 y el de 0-39 son los que tienen mayor y menor cantidad de fallecidos respectivamente. Ademas podemos observar un aumento pronunciado en el crecimiento de las curvas en todos los rangos etarios entre Junio y la mitad de Julio, finalizando con un peak que se puede apreciar graficamente como una linea casi vertical. Despues de este periodo, si podemos ver que el crecimiento para los grupos 60-69, 70-79 y 80-89 es mayor con respecto al resto de rangos etarios.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [11]:
casos_confirmados = alt.Chart(confirmados).mark_line(size=3).encode(
    x=alt.X('fecha', axis=alt.Axis(title='Mes')),
    y=alt.Y('sum(casos_confirmados)', axis=alt.Axis(title='Casos confirmados')),
    tooltip=[alt.Tooltip('sum(casos_confirmados)',title='Cantidad de casos confirmados'), alt.Tooltip('fecha', title='Fecha actual')]
).properties(
    title='Evolucion temporal de casos confirmados',
    width=350,
    height=300
)

fallecimientos = alt.Chart(fallecidos).mark_line(size=3).encode(
    x=alt.X('fecha', axis=alt.Axis(title='Mes')),
    y=alt.Y('sum(fallecidos)', axis=alt.Axis(title='Fallecimientos')),
    tooltip=[alt.Tooltip('sum(fallecidos)',title='Cantidad de casos confirmados'), alt.Tooltip('fecha', title='Fecha actual')]
).properties(title='Evolucion de fallecidos', width=350, height=300)


casos_activos = alt.Chart(activos).mark_line(size=3).encode(
    x=alt.X('fecha:T', axis=alt.Axis(title='Mes')),
    y=alt.Y('sum(casos_activos)', axis=alt.Axis(title='Casos activos')),
    tooltip=[alt.Tooltip('sum(casos_activos)',title='Cantidad de casos confirmados'), alt.Tooltip('fecha', title='Fecha actual')]
).properties(title='Evolucion de casos activos', width=350, height=300)


casos_confirmados | fallecimientos | casos_activos

alt.HConcatChart(...)

**Comentarios:** Del primer grafico podemos notar que la pendiente siempre va a ser positiva porque estamos considerando la cantidad total de casos confirmados, es importante notar que en mayo comienza el aumento significativo de casos con el peak a mediados de junio, en los otros graficos podemos ver que a partir de junio aumento mucho la cantidad de fallecidos y que la cantidad de casos activos tambien sube mucho. Luego de ese periodo, para el grafico de casos activos vemos que la curva se relaja y la cantidad de casos confirmados y fallecidos tambien disminuye su pendiente, que a fin de cuentas nos dice sobre el ritmo al que aumentan los casos confirmados/fallecidos.

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [57]:
line = alt.Chart(confirmados).mark_line(size=3).encode(
    x=alt.X('fecha:T', title='Mes'),
    y=alt.Y('mean(tasa):Q', title='Tasa'),
    color=alt.Color('region', title='Region:'),
    tooltip=alt.Tooltip('region',title='Region')
).properties(
    title='Evolucion de la tasa de incidencia por region',
    width=500,
    height=400


)

line

alt.Chart(...)

**Comentarios:** Podemos ver (poniendo el mouse encima de las curvas) que la region con mayor tasa de incidencia es Tarapacá, además identificamos un piño de regiones que no tuvieron una subida tan pronunciada como las regiones que se ven más arriba, en particular "Arica y Parinacota" y "Magallanes y de la antártida Chilena" tuvieron peaks mas retardados, en Agosto y Octubre respectivamente.

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [73]:
abril = alt.Chart(confirmados.loc[lambda x: confirmados['fecha']==start_date ]).mark_point().encode(
    x=alt.X('casos_confirmados', title='Casos confirmados'),
    y=alt.Y('mean(tasa)', title= 'Tasa'),
    color=alt.Color('region', title='Region:'),
    tooltip=alt.Tooltip('comuna',title='Comuna'),
    #size=alt.Size('poblacion', title='Poblacion:')
).properties(title='13 de Abril del 2020', width=500, height=500)


noviembre = alt.Chart(confirmados.loc[lambda x: confirmados['fecha']==end_date ]).mark_point().encode(
    x=alt.X('casos_confirmados', title='Casos confirmados'),
    y=alt.Y('mean(tasa)', title= 'Tasa'),
    color=alt.Color('region', title='Region:'),
    tooltip=alt.Tooltip('comuna',title='Comuna'),
    #size=alt.Size('poblacion', title='Poblacion')
).properties(title='6 de Noviembre del 2020', width=500, height=500)

abril | noviembre

alt.HConcatChart(...)

**Comentarios:** La conclusión que puedo sacar es que la tendencia es que las tasas por regiones apunta al crecimiento, si ponemos el mouse encima de los puntos mas alejados verticalmente de las aglomeraciones, nos damos cuenta que suelen ser comunas con poca cantidad de habitantes y esto se debe a la definicion de la tasa, pues 1000 casos confirmados valen (en terminos de tasa) mas para una poblacion de 5000 personas que para uno de 20000. Por otro lado, los puntos que se alejan horizontalmente de las aglomeraciones son comunas con alta cantidad de poblacion, y por lo tanto a pesar de que tienen una buena cantidad de casos confirmados, no termina siendo tan significativo en terminos de la tasa. Es por este analisis que sí considero útil que el tamaño de los círculos sea proporcional a la población de la región, grafico que puede verse abajito:

In [74]:
abril = alt.Chart(confirmados.loc[lambda x: confirmados['fecha']==start_date ]).mark_point().encode(
    x=alt.X('casos_confirmados', title='Casos confirmados'),
    y=alt.Y('mean(tasa)', title= 'Tasa'),
    color=alt.Color('region', title='Region:'),
    tooltip=alt.Tooltip('comuna',title='Comuna'),
    size=alt.Size('poblacion', title='Poblacion:')
).properties(title='13 de Abril del 2020', width=500, height=500)


noviembre = alt.Chart(confirmados.loc[lambda x: confirmados['fecha']==end_date ]).mark_point().encode(
    x=alt.X('casos_confirmados', title='Casos confirmados'),
    y=alt.Y('mean(tasa)', title= 'Tasa'),
    color=alt.Color('region', title='Region:'),
    tooltip=alt.Tooltip('comuna',title='Comuna'),
    size=alt.Size('poblacion', title='Poblacion')
).properties(title='6 de Noviembre del 2020', width=500, height=500)

abril | noviembre

alt.HConcatChart(...)

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [78]:
casos_activos = alt.Chart(activos).mark_line(size=3).encode(
    x=alt.X('fecha:T', axis=alt.Axis(title='Mes')),
    y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos')),
    color=alt.Color('comuna', title='Comuna:')
).properties(title='Evolucion de casos activos', width=500, height=400)

casos_activos

alt.Chart(...)

In [75]:
casos_activos = alt.Chart(activos).mark_line().encode(
    x=alt.X('fecha:T', axis=alt.Axis(title='Mes')),
    y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos')),
    color=alt.Color('comuna', title='Comuna'),
    column='region',
    tooltip=[alt.Tooltip('comuna',title='Comuna'), alt.Tooltip('fecha', title='Fecha')]
).properties(title='Evolucion de casos activos', width=350, height=300)

casos_activos

alt.Chart(...)

**Comentarios:** Pros:
Grafico 1: Es facil identificar las comunas con mayor cantidad de casos activos (colocando el mouse encima!) en determinada fecha

Grafico 2: Podemos identificar las comunas con mayor cantidad de casos activos, pero por region. Visualmente se ve notablemente mas ordenado.

Contras:
Grafico 1: Se pierde mucha informacion por la cantidad de datos en tan poco espacio, no podemos diferenciar informacion por regiones.

Grafico 2: Hacer comparaciones visualmente entre regiones puede resultar impreciso.

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [16]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
}

In [79]:
fallecidos['region']=fallecidos['region'].replace(region_names)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [17]:
import geopandas as gpd
from pathlib import Path

In [18]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

,Region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


In [19]:
type(regiones)

geopandas.geodataframe.GeoDataFrame

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [20]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

alt.Chart(...)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [21]:
casos_geo = (
    # FIX ME #
)

casos_geo.head()

AttributeError: 'tuple' object has no attribute 'head'

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = # FIX ME #
    
    chart = alt.Chart(data).mark_geoshape().encode(
        # FIX ME #
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [ ]:
col_widget = # FIX ME #

In [ ]:
fecha_widget = # FIX ME #

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** RESPONDE AQUÍ